In [1]:
# MAKE SURE THAT THE REPOSITORY ROOT IS IN THE PYTHONPATH
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

MODEL_FILE = "online_model/files/CNN_051620_SurrogateModel.h5"


# Scaling objects
Scaling objects are helpful because they allow us to generalize the model architecture to use any scale type preprocessing desired without having to construct a new model with embedded methods. In this case, the scale object is loaded and passed to the model as a parameters. Below is an example of an implementation for the MySurrogateModel example:

In [3]:
from online_model.model.surrogate_model import Scaler, load_model_info
model_info = load_model_info(MODEL_FILE)

# PRINT INFO CONTAINED IN THE MODEL FILE
print(model_info.keys())

Using TensorFlow backend.


dict_keys(['JSON', 'backend', 'bins', 'input_names', 'input_offsets', 'input_ordering', 'input_ranges', 'input_scales', 'input_units', 'keras_version', 'layer_names', 'lower', 'ndim', 'output_names', 'output_offsets', 'output_ordering', 'output_scales', 'output_units', 'type', 'upper'])


# Using the Scaler base class

The Scaler class is a base class that requires two methods to be initialized: transform (for input scaling) and inver_transform (for output scaling). This naming scheme is consistent with scikit-learn preprocessing scalers. Otherwise, the implementation is up the the discretion of the user.

The separation of the scaler from the model class is helpful because it allows for the separation of model execution logic and because scalers become portable. For instance, if using scikit-learn's MinMaxScaler, you could pickle the object, load the instance you used directly, and plug into the model.


In [5]:
class MyScaler(Scaler):
    def __init__(self, input_scales, input_offsets, output_scales,
                 output_offsets, model_value_min, model_value_max, 
                 image_input_scales, image_output_scales, 
                 n_scalar_vars, image_shape):
        
        self.input_scales = input_scales
        self.input_offsets = input_offsets
        self.output_scales = output_scales
        self.output_offsets = output_offsets
        self.model_value_min = model_value_min
        self.model_value_max = model_value_max
        self.n_scalar_vars = n_scalar_vars
        self.image_input_scales = image_input_scales
        self.image_output_scales = image_output_scales
        self.image_shape = image_shape
        
    # MUST OVERWRITE
    def transform(self, values, scale_type):
        if scale_type == "image":
            data_scaled = values / self.image_input_scale
            
        elif scale_type == "scalar":
            data_scaled = self.model_value_min + (
                (input_values - self.input_offsets[0 : self.n_scalar_vars])
                * (self.model_value_max - self.model_value_min)
                / self.input_scales)
        
        return data_scaled
    
    # MUST OVERWRITE
    def inverse_transform(self, values, scale_type):
        if scale_type == "image":
            data_unscaled = image_values * self.image_output_scale
        
        elif scale_type == "scalar":
            # reshape values
            
            values = values.reshape(values.shape[0], self.image_shape)    
    
            data_unscaled = (
                (input_values - self.min_value)
                * (self.input_scales[: self.n_scalar_vars])
                / (self.model_value_max - self.model_value_min)
            ) + self.input_offsets[: self.n_scalar_vars]
        
            data_unscaled = data_unscaled.reshape(self.image_shape)
            
        return data_unscaled

# MyScalar should be initialized with model info provided with the file:

In [ ]:
# get model info from file and initialize appropriately
model_info = load_model_info(MODEL_FILE)

# prepare info necessary to initialize
image_input_scales = model_info["input_scales"][-1]
image_output_scales = model_info["output_scales"][-1]
image_offset = model_info["output_offsets"][-1]
output_scales = model_info["output_scales"][:-1]
output_offsets = model_info["output_offsets"][:-1]
n_scalar_vars = len(model_info["input_ordering"])
n_scalar_outputs = len(model_info["output_ordering"])
input_scales = model_info["input_scales"][: n_scalar_vars]
input_offsets = model_info["input_offsets"][: n_scalar_vars]
model_value_min = model_info["lower"]
model_value_max = model_info["upper"]
image_shape = (model_info["bins"][0], model_info["bins"][1])

# Create instance of scaler object
my_scaler_obj = MyScaler(input_scales, input_offsets, output_scales,
                 output_offsets, model_value_min, model_value_max, 
                 image_input_scales, image_output_scales, 
                 n_scalar_vars, image_shape)

In [6]:
# CREATE A NEW MODEL CLASS
# import base classes
from online_model.model.surrogate_model import (SurrogateModel, 
                                                ReconstructedScaler,
                                                apply_temporary_ordering_patch)

# Using the SurrogateModel base class

The SurrogateModel class is the base class used for creating the server/model interface. The session and graph set up in the base class are necessary for running the model in a thread-safe manner. 

SurrogateModel provides an abstract method: `predict`. This means that the `predict` function MUST be implemented by the any class that inherits from this class. This is important because it allows us to generalize the server callbacks to call `predict` against any model implementation.

In [5]:
class MySurrogateModel(SurrogateModel):
    
    def __init__(self, model_file, scaler, stock_image_input):
        # Below line does all the work in loading + setting up 
        # the model session by calling the __init__ method from
        # the SurrogateModel class
        super(MySurrogateModel, self).__init__(model_file)
        self.model_file = model_file
        self.scaler = scaler
        self.stock_image_input = stock_image_input
        
        
        # can use the utility function to load the model info
        # and populate needed info
        model_info = load_model_info(model_file)
        self.ndim = model_info["ndim"]
        self.output_ordering = len(model_info["output_ordering"])
        
        
        # TEMPORARY PATCH FOR INPUT/OUTPUT REDUNDANT VARS
        self.input_ordering = apply_temporary_ordering_patch(
            self.input_ordering, "in")
        self.output_ordering = apply_temporary_ordering_patch(
            self.input_ordering, "out")
    
    # no need for an evaluate method, 
    # can handle everything in predict
    def predict(self, input_values):
        
        # scale inputs
        image_input = self.scaler.transform(np.array(input_values["image"]),
                                            "image")
        other_inputs = np.array([settings[key] for key in 
                                self.input_ordering])
        other_inputs = self.scaler.transform(other_inputs, "scalar")
        
        # now that this is scaled, we access the model attribute loaded by the 
        # SurrogateModel base class
        predicted_output = self.model.predict(
            [inputs_image_scaled, inputs_scalar_scaled]
        )
        
        # process the model output
        image_output = np.array(predicted_output[0])
        scalar_output = predicted_output[1]
        
        
        # unscale 
        image_output = self.scaler.inverse_transform(image_output, "image")
        scalar_output = self.scaler.inverse_transform(scalar_output, "scalar")
        
        # select extents
        extent_output = scalar_output[
            :, int(len(self.output_ordering) - self.ndim[0]) :
        ]
        
        # Now, format for server use
        formatted_output = dict(zip(self.output_ordering, scalar_output.T))
        formatted_output["extents"] = extent_output
        formatted_output["image"] = image_output
        
        

        
        

            